## Can RL be used for Classification?
*Let's find out...*

In [ ]:
import time
import gym
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from baselines import deepq
from baselines import bench
from baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

### Load MNIST Data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = keras.utils.to_categorical(y_test, num_classes)

### Keras Baseline

In [ ]:
def keras_train(batch_size=32, epochs=2):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(num_classes, activation="softmax")
        ]
    )

    model.summary()

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    start_time = time.time()
    model.fit(x_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    end_time = time.time()

    score = model.evaluate(x_test, y_test_one_hot, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    print("Training Time:", end_time - start_time)
    
keras_train()

### Create RL Interface (gym.Env)

In [ ]:

class MnistEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = gym.spaces.Box(low=0, high=1,
                                                shape=(28, 28, 1),
                                                dtype=np.float32)

        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs


### Train Classifier Using DQN

In [ ]:


def mnist_dqn():
    logger.configure(dir='./logs/mnist_dqn', format_strs=['stdout', 'tensorboard'])
    env = MnistEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.learn(
        env,
        "mlp",
        num_layers=1,
        num_hidden=64,
        activation=tf.nn.relu,
        hiddens=[32],
        dueling=True,
        lr=1e-4,
        total_timesteps=int(1.2e5),
        buffer_size=10000,
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        train_freq=4,
        learning_starts=10000,
        target_network_update_freq=1000,
    )

    model.save('dqn_mnist.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = mnist_dqn()
print("DQN Training Time:", time.time() - start_time)

In [ ]:
def mnist_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = MnistEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

mnist_dqn_eval(dqn_model)

### Train Classifier using PPO

In [ ]:
def mnist_ppo():
    logger.configure(dir='./logs/mnist_ppo', format_strs=['stdout', 'tensorboard'])
    env = DummyVecEnv([lambda: bench.Monitor(MnistEnv(images_per_episode=1), logger.get_dir())])

    model = ppo2.learn(
        env=env,
        network='mlp',
        num_layers=2,
        num_hidden=64,
        nsteps=32,
        total_timesteps=int(1.2e5),
        seed=int(time.time()))
    
    return model

start_time = time.time()
ppo_model = mnist_ppo()
print("PPO Training Time:", time.time() - start_time)


In [ ]:
def mnist_ppo_eval(ppo_model):
    attempts, correct = 0,0

    env = DummyVecEnv([lambda: MnistEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)])
    
    try:
        while True:
            obs, done = env.reset(), [False]
            while not done[0]:
                obs, rew, done, _ = env.step(ppo_model.step(obs[None])[0])

                attempts += 1
                if rew[0] > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

mnist_ppo_eval(ppo_model)

### Conclusions?

**Facts**
- All models used roughly the **same number of parameters**
- DQN model splits final layer into two for value and advantages heads
- PPO model adds a value head after the final layer
- All models are run for equivalent of **2 epochs**

**Results**
- Standard Supervised Learning
    - Training Time: 11.93s
    - Test Accuracy: 96.27%
    
- DQN
    - Training Time: 461.5s
    - Test Accuracy: 93.48%
    
- PPO
    - Training Time: 638.7s
    - Test Accuracy: 95.20%
    
**Remarks**

RL can be used for classification, but it's clearly not the optimal method and doesn't present any advantages over using standard supervised learning.